In [2]:
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import csv
import os

In [3]:
log_reg_model = pickle.load(open("ml_models/models_weights/logistic_regression/logistic_regression_93tr_91ts.sav", "rb"))
tf_idf_model = pickle.load(open("ml_models/models_weights/tf_idf/tf_idf_vectorizer_50000_fetures.pickle", "rb"))

In [7]:
path_to_read = 'csv_down_files/preprocessed/COVID19-tweetID-2020-01/' # January Month of 2020 
path_to_save = 'csv_down_files/classifed_tweeets_of_each_file/COVID19-tweetID-2020-01/csv/' # Classify each file tweets

# Small files functions

In [9]:
def classify_tweets_of_file(file_path, df_file):  
    tweets = list(df_file['full_text'])
    tweets_features = tf_idf_model.transform(tweets)
    tweets_features_array = tweets_features.toarray()
    predict = log_reg_model.predict(tweets_features_array)
    classifed_dict    = {'tweet_text': tweets, 'class': predict}
    classifed_data    = pd.DataFrame(classifed_dict)
    return classifed_data



def read_direction_to_classify(direction_path, path_to_save):
    try:
        all_path_files                          = os.listdir(direction_path)
        cls = []
        for file_path in all_path_files:
            file_path                           = direction_path + file_path
            print((file_path))
            df_file   = pd.read_csv(file_path, error_bad_lines=False, encoding='utf-8', engine='python')
            cls.extend(df_file['class'])
#             classify_tweets                     = classify_tweets_of_file(file_path, df_file)
#             file_path                           = file_path.split('/')
#             file_name                           = file_path[3]
#             path_to_save_excel = path_to_save
#             path_to_save_csv = path_to_save + "csv/" + file_name
            
#             path_to_save_excel = path_to_save_excel + "excel/" + file_name
#             path_to_save_excel = path_to_save_excel.replace(".csv", '.xlsx')
#             classify_tweets.to_csv(path_to_save_csv, index=False)
#             classify_tweets.to_excel(path_to_save_excel, index=False)
    except Exception as e:
        file = open("logs/direction_and_file_handleing_file.log","+a")
        file.write("This error related to function read_direction_preprocess of direction_and_file_handleing file in direction config_files n"
                   + str(e) + "\n" + "#" *99 + "\n") # "#" *99 as separated lines
    return cls

cls = read_direction_to_classify(path_to_save, path_to_save)
len(cls)

csv_down_files/classifed_tweeets_of_each_file/COVID19-tweetID-2020-01/csv/COVID19-tweetID-2020-01-14.csv
csv_down_files/classifed_tweeets_of_each_file/COVID19-tweetID-2020-01/csv/COVID19-tweetID-2020-01-01.csv
csv_down_files/classifed_tweeets_of_each_file/COVID19-tweetID-2020-01/csv/COVID19-tweetID-2020-01-02.csv
csv_down_files/classifed_tweeets_of_each_file/COVID19-tweetID-2020-01/csv/COVID19-tweetID-2020-01-03.csv
csv_down_files/classifed_tweeets_of_each_file/COVID19-tweetID-2020-01/csv/COVID19-tweetID-2020-01-04.csv
csv_down_files/classifed_tweeets_of_each_file/COVID19-tweetID-2020-01/csv/COVID19-tweetID-2020-01-05.csv
csv_down_files/classifed_tweeets_of_each_file/COVID19-tweetID-2020-01/csv/COVID19-tweetID-2020-01-06.csv
csv_down_files/classifed_tweeets_of_each_file/COVID19-tweetID-2020-01/csv/COVID19-tweetID-2020-01-07.csv
csv_down_files/classifed_tweeets_of_each_file/COVID19-tweetID-2020-01/csv/COVID19-tweetID-2020-01-08.csv
csv_down_files/classifed_tweeets_of_each_file/COVID19-t

304418

# Big files functions

In [5]:
# def big_file_classify(df_file, file_name, path_to_save):
#     tweets = list(df_file['full_text'])
#     df_len = len(df_file)
#     to_index = 0
#     prdict_all = []
#     while to_index <= df_len:
#         part_of_tweets = tweets[to_index: to_index + 10000]
#         to_index = to_index + 10000
#         tweets_features = tf_idf_model.transform(part_of_tweets)
#         tweets_features_array = tweets_features.toarray()
#         predict = log_reg_model.predict(tweets_features_array)
#         prdict_all.extend(predict)
#     classifed_dict    = {'tweet_text': tweets, 'class': prdict_all}
#     classifed_data    = pd.DataFrame(classifed_dict)
#     path_to_save_csv = path_to_save + "csv/" + file_name
#     file_name = file_name.replace('.csv', '.xlsx')
#     path_to_save_excel = path_to_save + "excel/" + file_name
#     classifed_data.to_csv(path_to_save_csv, index=False)
#     classifed_data.to_excel(path_to_save_excel, index=False)
#     return True

In [6]:
# def read_direction_big_files(direction_path, path_to_save):
#     try:
#         all_path_files                          = os.listdir(direction_path)
#         for file_path in all_path_files:
#             file_path                           = direction_path + file_path
#             print((file_path))
#             df_file   = pd.read_csv(file_path, error_bad_lines=False, encoding='utf-8', engine='python')
            
#             file_path                           = file_path.split('/')
#             file_name                           = file_path[3]
#             big_file_classify(df_file, file_name, path_to_save)
#     except Exception as e:
#         file = open("logs/direction_and_file_handleing_file.log","+a")
#         file.write("This error related to function read_direction_preprocess of direction_and_file_handleing file in direction config_files n"
#                    + str(e) + "\n" + "#" *99 + "\n") # "#" *99 as separated lines
#     return True

In [7]:
# read_direction_big_files(path_to_read, path_to_save)

In [8]:
def get_all_tweets(direction_read):
    try:
        all_path_files              = os.listdir(direction_read)
        all_files_tweets            = []
        all_files_tweets_predicited = []
        
        for file_path in all_path_files:
            file_path           = direction_read + file_path
            df_file             = pd.read_csv(file_path, lineterminator='\n', error_bad_lines=False, encoding='utf-8')
            all_files_tweets    += list(df_file['tweet_text'])
            all_files_tweets_predicited    += list(df_file['class'])
        dicts_df                = {"tweet_text": all_files_tweets, 'class':all_files_tweets_predicited}
        df_file                 = pd.DataFrame(dicts_df)
              
    except Exception as e:
        file = open("logs/get_tweet_to_classify.log","+a")
        file.write("This error related to function get_random_number_of_tweets of get_tweet_to_classify file in direction config_files n"
                   + str(e) + "\n" + "#" *99 + "\n") # "#" *99 as separated lines
    return df_file


In [9]:
path = path_to_save + 'csv/'
df_file = get_all_tweets(path)

In [10]:
df_file.head()

,tweet_text,class
0,منظمه الصحه العالميه تدعو الصين التحقيق في مص...,1
1,الصحه العالميه ظهور كورونا جديد في الصين,1
2,الصحه العالميه ظهور كورونا جديد في الصين\...,1
3,وضع الاطفال الشاشات صباحا يزيد خطر تعرضهم لمشك...,0
4,الصحه العالميه تحذر ظهور كورونا جديد في الص...,1


In [18]:
import numpy as np
cls = np.array(cls)
cls[:10]

array([1, 1, 1, 0, 1, 1, 0, 0, 0, 0])

In [19]:
class_1 = np.sum(cls == 1)
class_1

128971

In [20]:
class_0 = np.sum(cls == 0)
class_0

175447

In [21]:
print("Number of coronavirus Tweets: ", class_1)
print("Number of other Tweets: ", class_0)

Number of coronavirus Tweets:  128971
Number of other Tweets:  175447


In [25]:
histogram = [0, 1]

labels= 'Class 1 Coronavirus', 'Class 0 General'
sizes = [class_1, class_0]
explode= (0, 0.1)  
fig1, ax1= plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90)
plt.title("Coronavirus Vs General Tweets")
plt.savefig('test2')